In [1]:
import numpy as np
import torch 
import yaml
import torch.nn as nn
import math 

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import src.models as models
import src.dataset as ds
import src.train_pcdae as train_pcdae
import src.inference_pcdae as inference
import src.inference_regressor as constraints
import src.data_preparation as data_prep

In [2]:

config_file = "config_regressor.yaml"
device = torch.device('cpu')

try:
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Error: The file 'config.yaml' was not found.")
    exit()
except yaml.YAMLError as e:
    print(f"Error parsing YAML file: {e}")
    exit()
    


In [3]:
def inference_test(seed):
    
    ##! fix the seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    ###! load data and trained model
    data_dict = data_prep.prepare_data_model(seed, config['data']["data_path"], config['training']["ratio_test_val_train"])
    X_test_scaled, y_test_scaled = data_dict['test']
    scaler_X, scaler_Y = data_dict['scalers']
    batch_size = config['training']['batch_size']

    test_dataset = ds.LTPDataset(X_test_scaled, y_test_scaled)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ###* define the PCDAE architecture
    input_size = config['model']['x_dim']
    output_size = config['model']['y_dim']
    hidden_sizes = config['model']['hidden_size']
    
    regressor = models.Regressor(input_size=input_size, output_size=output_size, hidden_sizes=hidden_sizes).to(device)
    regressor.load_state_dict(torch.load(f"models_regressor_saved/regressor_inference_{seed}.pth"))
    
    solver = constraints.ProjectionSolver(scaler_X, scaler_Y, x_dim=input_size, p_dim=output_size)
    y_pred, p_pred = solver.solve_batch(regressor, X_test_scaled[:,:], scaler_Y)
    
    ###! EBM sampling

    test_init_loss = 0
    test_T1_loss = 0
    test_T2_loss = 0
    
    solver = constraints.ProjectionSolver(scaler_X, scaler_Y, x_dim=input_size, p_dim=output_size)
    y_pred, p_pred = solver.solve_batch(regressor, X_test_scaled[:,:], scaler_Y)
    
    loss_net_pred = ((y_pred - y_test_scaled)**2).mean()
    loss_proj_pred = ((p_pred - y_test_scaled)**2).mean()
    
    constant_net = inference.get_constraints_scalar(torch.from_numpy(X_test_scaled), torch.from_numpy(y_pred), scaler_X, scaler_Y).pow(2)
    constant_proj = inference.get_constraints_scalar(torch.from_numpy(X_test_scaled), torch.from_numpy(p_pred), scaler_X, scaler_Y).pow(2)

    
    return np.sqrt(loss_net_pred), np.sqrt(loss_proj_pred), regressor, constant_net, constant_proj



In [4]:

# seed_vec = np.array([1, 41, 81, 121, 161, 201, 241, 281, 321, 361])
seed_vec = np.array([1])

rmse_net_vec = []
rmse_proj_vec = []

for i, seed in enumerate(seed_vec):
    
    val_net, val_proj, pcdae, constraint_net, constraint_proj = inference_test(seed)
    
    rmse_net_vec.append(val_net)
    rmse_proj_vec.append(val_proj)
    
    print("seed: ", seed)
    



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Item 0: Constraint Violation (Sum Abs) = 5.65e-11
Item 1: Constraint Violation (Sum Abs) = 5.73e-11
Item 2: Constraint Violation (Sum Abs) = 6.57e-16
Item 3: Constraint Violation (Sum Abs) = 8.33e-16
Item 4: Constraint Violation (Sum Abs) = 1.64e-16
Item 5: Constraint Violation (Sum Abs) = 6.20e-16
Item 6: Constraint Violation (Sum Abs) = 1.54e-11
Item 7: Constraint Violation (Sum Abs) = 0.00e+00
Item 8: Constraint Violation (Sum Abs) = 2.66e-16
Item 9: Constraint Violation (Sum Abs) = 6.35e-13
Item 10: Constraint Violation (Sum Abs) = 2.48e-11
Item 11: Constraint Violation (Sum Abs) = 9.45e-12
Item 12: Co

In [10]:

rmse_net_arr = np.array(rmse_net_vec)
rmse_proj_arr = np.array(rmse_proj_vec)


print(np.mean(rmse_net_vec))
print(np.mean(rmse_proj_vec))
print()

# print(np.sqrt(constraint_net))
print(np.sqrt(constraint_proj.mean()))

0.03747354148123894
0.037160208489565194

tensor(1.1588e-11, dtype=torch.float64)


In [6]:
# 1.4022851448245575
# 0.022653040582816645
# 0.022657518059754155

In [7]:
# 0.01311752153355084

In [8]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


print(count_parameters(pcdae))

11132


In [9]:
# 1.4364558830118268
# 0.031021168598093285
# 0.01793517113448239